https://jira.x5.ru/browse/CVMUPG-1046

In [1]:
name = 'CVMUPG-1046_danone'

In [2]:
import sys
import pandas as pd
import pyspark.sql.functions as F
import datetime
from datetime import timedelta
sys.path.append('/home/jovyan/glow-byte-filters-pyspark')
from logic_filters import * 
from segmentation import *
from spark import *

%load_ext autoreload
%autoreload 2

pd.options.display.float_format = '{:,.2f}'.format
pd.options.display.max_rows = 500

In [3]:
sc, spark = restart_spark(name, 100, executor_memory="3G", executor_cores=5, driver_memory="12G",
                          additional_params={"spark.sql.shuffle.partitions": "200"})
sc.setLogLevel('ERROR')

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
22/01/25 14:09:04 WARN cluster.YarnSchedulerBackend$YarnSchedulerEndpoint: Attempted to request executors before the AM has registered!


In [4]:
LOYALTY_CARDS = "hive_ssa_tc5.loyalty_card"
LOYALTY_CARDHOLDERS = "hive_ssa_tc5.loyalty_cardholder"
ACCOUNTS = "hive_ssa_tc5.account"
CVM5_GUESTS = "hive_cvm_acrm.cvm5_guests"

DIM_STORE = "hive_ssa_main.dim_store"
CHECKS_HEADERS = "hive_ssa_main.fct_rtl_txn"
CHECKS_ITEMS = "hive_ssa_main.fct_rtl_txn_item"
PRODUCTS = "hive_ssa_tc5.cvm_product"

### Выбираем гостей нужного юзкейса

In [5]:
usecase = ['cross', 'upgrade', 'ump']
dt = datetime.date.today()

In [6]:
customers_usecase = (spark
                     .table(CVM5_GUESTS)
                     .filter(F.to_date('calculation_dt') == dt)
                     .filter(F.col('usecase').isin(usecase))
                     .select('account_no', 'customer_rk')
                            )

In [7]:
customers_usecase.count() # 15 972 301

15972301

### Проверяем на доступность отобранных гостей в определенную дату

In [8]:
check_date = '2022-03-01'

In [9]:
seg_sms1 = (sms_channel_filters_glowbyte(spark=spark,
                                         guests=customers_usecase, 
                                         usecase_name=usecase[0], 
                                         check_date=check_date, 
                                         debug_mode=False)
                                    .select('customer_rk')
                                    .distinct()
           )

 Время выполнения: 0:01:55


In [10]:
seg_sms2 = (sms_channel_filters_glowbyte(spark=spark,
                                         guests=customers_usecase, 
                                         usecase_name=usecase[1], 
                                         check_date=check_date, 
                                         debug_mode=False)
                                    .select('customer_rk')
                                    .distinct()
           )

 Время выполнения: 0:01:51


In [11]:
seg_sms3 = (sms_channel_filters_glowbyte(spark=spark,
                                         guests=customers_usecase, 
                                         usecase_name=usecase[2], 
                                         check_date=check_date, 
                                         debug_mode=False)
                                    .select('customer_rk')
                                    .distinct()
           )

 Время выполнения: 0:02:06


In [12]:
seg_sms = seg_sms1.union(seg_sms2).union(seg_sms3).distinct()

In [13]:
seg_sms.write.parquet('temp1046_1', mode='overwrite')
seg_sms = spark.read.parquet('temp1046_1')

In [14]:
seg_sms.count() # 

9385079

## Отбор магазинов из таблицы Excel

In [15]:
stores = pd.read_excel('need_info_store.xlsx')

In [23]:
stores = stores['financial_unit_id'].tolist()

In [24]:
len(stores)

10612

In [27]:
plants = (spark.table(DIM_STORE)
            .filter(F.col('valid_to_dttm')== datetime.datetime(5999, 1, 1, 0, 0))
            .filter(F.col('financial_unit_id').isin(stores))
            .selectExpr('store_id as plant')
            .distinct()
            .toPandas()['plant']
            .tolist()
            )

len(plants)

12300

## Группы

### Собираем чеки этих гостей и оставляем тех, кто покупал продукты с synthetic_catalog_id

In [25]:
start_date1 = datetime.date(2021, 7, 16)
end_date1 = datetime.date(2021, 10, 15)
start_date2 = datetime.date(2021, 10, 16)
end_date2 = datetime.date(2022, 1, 15)

In [26]:
checks_headers = (spark.table(CHECKS_HEADERS)
                      .filter(F.col('rtl_txn_dt').between(start_date2, end_date2))
                      .filter((F.col('loyalty_card_no') != '') & (F.col('loyalty_card_no').isNotNull()))
                      .filter(F.col('financial_unit_format_dk') == 'D')
                      .filter(F.col('rtl_txn_cancel_flg') == 0)
                      .select('rtl_txn_id', 'loyalty_card_no', 'store_id')
                     )


In [28]:
checks_headers_tc5 = checks_headers.filter(F.col('store_id').isin(plants)) #оставили чеки только с нужными магазинами

In [29]:
loyalty_cards = (spark
                    .table(LOYALTY_CARDS)
                    .withColumnRenamed("loyalty_card_id", "loyalty_card_no")
                    .withColumnRenamed("loyalty_account_id", "account_no")
                    .withColumnRenamed("loyalty_account_acrm_id", "account_rk")
                    .select('account_no', 'loyalty_card_no')
                )
loyalty_cardholders = (spark
                        .table(LOYALTY_CARDHOLDERS)
                        .withColumnRenamed("loyalty_cardholder_acrm_id", "customer_rk")
                        .withColumnRenamed("loyalty_account_id", "account_no")
                        .select('account_no', 'customer_rk')
                      )
clients_info = loyalty_cards.join(loyalty_cardholders, on='account_no', how='inner')
clients_info = clients_info.join(seg_sms, on='customer_rk', how='inner')

In [30]:
checks_headers_tc5 = checks_headers_tc5.join(clients_info, on='loyalty_card_no') #оставили чеки только нужных гостей

In [31]:
checks_items = (spark.table(CHECKS_ITEMS) 
                    .withColumnRenamed('plu_id', 'plu_code')
                    .withColumnRenamed('turnover_no_vat_amt', 'zsalnovat')
                    .withColumnRenamed('turnover_vat_rub_amt', 'zsale_vat')
                    .withColumnRenamed('prime_cost_no_vat_amt', 'zcst_n')
                    .withColumnRenamed('turnover_base_uom_amt', 'base_qty')
                    .withColumnRenamed('discount_amt', 'zdiscount')
                    .withColumnRenamed('fact_regular_promo_flg', 'zpromofl')
                    .filter(F.col('rtl_txn_dt').between(start_date2, end_date2))
                    .filter((F.col('zsalnovat') >= 0) & (F.col('base_qty') >= 0) & (F.col('zcst_n') > 0)) #keep only correct data
                        .select('plu_code' #items id
                                , 'rtl_txn_id' #cheques id
                               )
                   )


In [133]:
plu_hierarchy_lvl_4_dk = ['FR0604001', 'FR0608005', 'FR0604002', 'FR0608001'] #сметана, молоко, кефир
# syntethic_category_id = [51,26,52]
# plu_brand_code = ['5727','M082','M083','5728']
plu_vendor_nm = ['Данон']
# plu_not_in = [4138521]

In [136]:
plu_brands = (spark
                 .table(PRODUCTS)
                 .filter(F.col('plu_hierarchy_lvl_4_dk').isin(plu_hierarchy_lvl_4_dk))
                 .filter(F.col('plu_vendor_nm').isin(plu_vendor_nm))
#                  .filter(F.col('syntethic_category_id').isin(syntethic_category_id))
#                  .filter(~F.col('plu_id').isin(plu_not_in))
                 .select('plu_brand_hierarchy_lvl_nm')
                 .distinct()
                 .toPandas()
                )

In [137]:
plu_brands

,plu_brand_hierarchy_lvl_nm
0,АКТИВИА
1,ПЕТМОЛ
2,None
3,КНЯЗЬ ВЕЖИЧ
4,СЕЛО ЛУГОВОЕ
5,БИО БАЛАНС
6,EDELWEISS
7,ДАНОН ЭКОНОМНЫЙ
8,ДЛЯ ВСЕЙ СЕМЬИ
9,DANONE


In [73]:
plu_codes_cat = (spark
                 .table(PRODUCTS)
                 .filter(F.col('plu_hierarchy_lvl_4_dk').isin(plu_hierarchy_lvl_4_dk))
                 .filter(F.col('plu_vendor_nm').isin(plu_vendor_nm))
#                  .filter(F.col('syntethic_category_id').isin(syntethic_category_id))
#                  .filter(~F.col('plu_id').isin(plu_not_in))
                 .select('plu_id')
                 .distinct()
                 .toPandas()['plu_id']
                 .tolist()
                )

In [79]:
checks_items = checks_items.filter(F.col('plu_code').isin(plu_codes_cat)) # только чеки с нужными plu

In [80]:
checks_tc5 = checks_items.join(checks_headers_tc5, 'rtl_txn_id', how='inner')

In [81]:
# те, кто покупал во 2 период даниссимо
seg1 = (checks_tc5.select('customer_rk')
                  .distinct()
      )

In [82]:
# те, кто не покупал во 2 период даниссимо
seg2 = seg_sms.join(seg1, on='customer_rk', how='left_anti')

In [83]:
checks_headers2 = (spark.table(CHECKS_HEADERS)
                      .filter(F.col('rtl_txn_dt').between(start_date1, end_date1))
                      .filter((F.col('loyalty_card_no') != '') & (F.col('loyalty_card_no').isNotNull()))
                      .filter(F.col('financial_unit_format_dk') == 'D')
                      .filter(F.col('rtl_txn_cancel_flg') == 0)
                      .select('rtl_txn_id', 'loyalty_card_no', 'store_id')
                     )


In [84]:
checks_headers_tc5_2 = checks_headers2.filter(F.col('store_id').isin(plants)) #оставили чеки только с нужными магазинами

In [85]:
loyalty_cards2 = (spark
                    .table(LOYALTY_CARDS)
                    .withColumnRenamed("loyalty_card_id", "loyalty_card_no")
                    .withColumnRenamed("loyalty_account_id", "account_no")
                    .withColumnRenamed("loyalty_account_acrm_id", "account_rk")
                    .select('account_no', 'loyalty_card_no')
                )
loyalty_cardholders2 = (spark
                        .table(LOYALTY_CARDHOLDERS)
                        .withColumnRenamed("loyalty_cardholder_acrm_id", "customer_rk")
                        .withColumnRenamed("loyalty_account_id", "account_no")
                        .select('account_no', 'customer_rk')
                      )
clients_info2 = loyalty_cards2.join(loyalty_cardholders2, on='account_no', how='inner')
clients_info2 = clients_info2.join(seg_sms, on='customer_rk', how='inner')

In [86]:
checks_headers_tc5_2 = checks_headers_tc5_2.join(clients_info2, on='loyalty_card_no') #оставили чеки только нужных гостей

In [87]:
checks_items2 = (spark.table(CHECKS_ITEMS) 
                    .withColumnRenamed('plu_id', 'plu_code')
                    .withColumnRenamed('turnover_no_vat_amt', 'zsalnovat')
                    .withColumnRenamed('turnover_vat_rub_amt', 'zsale_vat')
                    .withColumnRenamed('prime_cost_no_vat_amt', 'zcst_n')
                    .withColumnRenamed('turnover_base_uom_amt', 'base_qty')
                    .withColumnRenamed('discount_amt', 'zdiscount')
                    .withColumnRenamed('fact_regular_promo_flg', 'zpromofl')
                    .filter(F.col('rtl_txn_dt').between(start_date1, end_date1))
                    .filter((F.col('zsalnovat') >= 0) & (F.col('base_qty') >= 0) & (F.col('zcst_n') > 0)) #keep only correct data
                        .select('plu_code' #items id
                                , 'rtl_txn_id' #cheques id
                               )
                   )


In [88]:
checks_items2 = checks_items2.filter(F.col('plu_code').isin(plu_codes_cat)) # только чеки с нужными plu

In [89]:
checks_tc5_2 = checks_items2.join(checks_headers_tc5_2, 'rtl_txn_id', how='inner')

In [90]:
# те, кто покупал в 1 период даниссимо
seg3 = (checks_tc5_2.select('customer_rk')
                  .distinct()
      )

In [91]:
# те, кто не покупал во 2 период даниссимо, хотя раньше покупали
seg = seg3.join(seg2, on='customer_rk', how='inner')

In [92]:
seg.write.parquet('temp1046_seg1', mode='overwrite')
seg = spark.read.parquet('temp1046_seg1')

In [93]:
seg.count()

2182959

In [94]:
seg_pd1 = seg.toPandas()

In [95]:
len(seg_pd1)

2182959

In [96]:
seg_no = 'seg1'

In [97]:
seg_pd1.to_csv(name + '_' + seg_no + '_All_uc' + str(dt) + '.csv', index=False)

In [101]:
seg_pd1['customer_rk'] = seg_pd1['customer_rk'].astype('str')

In [102]:
cross, upgrade, ump = get_usecases(seg_pd1)

In [104]:
print('cross: {}, upgrade: {}, ump: {} (all: {})'.format(len(cross), len(upgrade), len(ump), len(cross)+len(upgrade)+len(ump)))

cross: 891293, upgrade: 806867, ump: 484799 (all: 2182959)


### Собираем чеки этих гостей и оставляем тех, кто покупал продукты с synthetic_catalog_id

In [105]:
start_date1 = datetime.date(2021, 7, 16)
end_date1 = datetime.date(2021, 10, 15)
start_date2 = datetime.date(2021, 10, 16)
end_date2 = datetime.date(2022, 1, 15)

In [106]:
checks_headers = (spark.table(CHECKS_HEADERS)
                      .filter(F.col('rtl_txn_dt').between(start_date2, end_date2))
                      .filter((F.col('loyalty_card_no') != '') & (F.col('loyalty_card_no').isNotNull()))
                      .filter(F.col('financial_unit_format_dk') == 'D')
                      .filter(F.col('rtl_txn_cancel_flg') == 0)
                      .select('rtl_txn_id', 'loyalty_card_no', 'store_id')
                     )


In [107]:
checks_headers_tc5 = checks_headers.filter(F.col('store_id').isin(plants)) #оставили чеки только с нужными магазинами

In [108]:
loyalty_cards = (spark
                    .table(LOYALTY_CARDS)
                    .withColumnRenamed("loyalty_card_id", "loyalty_card_no")
                    .withColumnRenamed("loyalty_account_id", "account_no")
                    .withColumnRenamed("loyalty_account_acrm_id", "account_rk")
                    .select('account_no', 'loyalty_card_no')
                )
loyalty_cardholders = (spark
                        .table(LOYALTY_CARDHOLDERS)
                        .withColumnRenamed("loyalty_cardholder_acrm_id", "customer_rk")
                        .withColumnRenamed("loyalty_account_id", "account_no")
                        .select('account_no', 'customer_rk')
                      )
clients_info = loyalty_cards.join(loyalty_cardholders, on='account_no', how='inner')
clients_info = clients_info.join(seg_sms, on='customer_rk', how='inner')

In [109]:
checks_headers_tc5 = checks_headers_tc5.join(clients_info, on='loyalty_card_no') #оставили чеки только нужных гостей

In [110]:
checks_items = (spark.table(CHECKS_ITEMS) 
                    .withColumnRenamed('plu_id', 'plu_code')
                    .withColumnRenamed('turnover_no_vat_amt', 'zsalnovat')
                    .withColumnRenamed('turnover_vat_rub_amt', 'zsale_vat')
                    .withColumnRenamed('prime_cost_no_vat_amt', 'zcst_n')
                    .withColumnRenamed('turnover_base_uom_amt', 'base_qty')
                    .withColumnRenamed('discount_amt', 'zdiscount')
                    .withColumnRenamed('fact_regular_promo_flg', 'zpromofl')
                    .filter(F.col('rtl_txn_dt').between(start_date2, end_date2))
                    .filter((F.col('zsalnovat') >= 0) & (F.col('base_qty') >= 0) & (F.col('zcst_n') > 0)) #keep only correct data
                        .select('plu_code' #items id
                                , 'rtl_txn_id' #cheques id
                               )
                   )


In [111]:
plu_codes_cat = [2102454,3142744,97452,4121609,3172778,4121592,3397795,3601971,3432833,2085981,3400885,2093081,
3667086,4121607,4045383,3150905,3601973,3913333,3605310,4120901,3352331,3173204,4121582,3601972,3181748,4045385,
4045460,3601974,4000350,3667087,3142746,3605407,2093082]

In [112]:
checks_items = checks_items.filter(F.col('plu_code').isin(plu_codes_cat)) # только чеки с нужными plu

In [113]:
checks_tc5 = checks_items.join(checks_headers_tc5, 'rtl_txn_id', how='inner')

In [114]:
# те, кто покупал во 2 период даниссимо
seg1 = (checks_tc5.select('customer_rk')
                  .distinct()
      )

In [115]:
# те, кто не покупал во 2 период даниссимо
seg2 = seg_sms.join(seg1, on='customer_rk', how='left_anti')

In [116]:
checks_headers2 = (spark.table(CHECKS_HEADERS)
                      .filter(F.col('rtl_txn_dt').between(start_date1, end_date1))
                      .filter((F.col('loyalty_card_no') != '') & (F.col('loyalty_card_no').isNotNull()))
                      .filter(F.col('financial_unit_format_dk') == 'D')
                      .filter(F.col('rtl_txn_cancel_flg') == 0)
                      .select('rtl_txn_id', 'loyalty_card_no', 'store_id')
                     )


In [117]:
checks_headers_tc5_2 = checks_headers2.filter(F.col('store_id').isin(plants)) #оставили чеки только с нужными магазинами

In [118]:
loyalty_cards2 = (spark
                    .table(LOYALTY_CARDS)
                    .withColumnRenamed("loyalty_card_id", "loyalty_card_no")
                    .withColumnRenamed("loyalty_account_id", "account_no")
                    .withColumnRenamed("loyalty_account_acrm_id", "account_rk")
                    .select('account_no', 'loyalty_card_no')
                )
loyalty_cardholders2 = (spark
                        .table(LOYALTY_CARDHOLDERS)
                        .withColumnRenamed("loyalty_cardholder_acrm_id", "customer_rk")
                        .withColumnRenamed("loyalty_account_id", "account_no")
                        .select('account_no', 'customer_rk')
                      )
clients_info2 = loyalty_cards2.join(loyalty_cardholders2, on='account_no', how='inner')
clients_info2 = clients_info2.join(seg_sms, on='customer_rk', how='inner')

In [119]:
checks_headers_tc5_2 = checks_headers_tc5_2.join(clients_info2, on='loyalty_card_no') #оставили чеки только нужных гостей

In [120]:
checks_items2 = (spark.table(CHECKS_ITEMS) 
                    .withColumnRenamed('plu_id', 'plu_code')
                    .withColumnRenamed('turnover_no_vat_amt', 'zsalnovat')
                    .withColumnRenamed('turnover_vat_rub_amt', 'zsale_vat')
                    .withColumnRenamed('prime_cost_no_vat_amt', 'zcst_n')
                    .withColumnRenamed('turnover_base_uom_amt', 'base_qty')
                    .withColumnRenamed('discount_amt', 'zdiscount')
                    .withColumnRenamed('fact_regular_promo_flg', 'zpromofl')
                    .filter(F.col('rtl_txn_dt').between(start_date1, end_date1))
                    .filter((F.col('zsalnovat') >= 0) & (F.col('base_qty') >= 0) & (F.col('zcst_n') > 0)) #keep only correct data
                        .select('plu_code' #items id
                                , 'rtl_txn_id' #cheques id
                               )
                   )


In [121]:
checks_items2 = checks_items2.filter(F.col('plu_code').isin(plu_codes_cat)) # только чеки с нужными plu

In [122]:
checks_tc5_2 = checks_items2.join(checks_headers_tc5_2, 'rtl_txn_id', how='inner')

In [123]:
# те, кто покупал в 1 период даниссимо
seg3 = (checks_tc5_2.select('customer_rk')
                  .distinct()
      )

In [124]:
# те, кто не покупал во 2 период даниссимо, хотя раньше покупали
seg = seg3.join(seg2, on='customer_rk', how='inner')

In [125]:
seg.write.parquet('temp1046_seg2', mode='overwrite')
seg = spark.read.parquet('temp1046_seg2')

In [126]:
seg.count()

767549

In [127]:
seg_pd2 = seg.toPandas()

In [128]:
seg_no = 'seg2'

In [129]:
seg_pd2.to_csv(name + '_' + seg_no + '_All_uc' + str(dt) + '.csv', index=False)

In [130]:
seg_pd2['customer_rk'] = seg_pd2['customer_rk'].astype('str')

In [131]:
cross, upgrade, ump = get_usecases(seg_pd2)

In [132]:
print('cross: {}, upgrade: {}, ump: {} (all: {})'.format(len(cross), len(upgrade), len(ump), len(cross)+len(upgrade)+len(ump)))

cross: 313097, upgrade: 285276, ump: 169176 (all: 767549)
